### Name: Serena Yang
### DSC 478 Homework 4

In [1]:
path = "/Users/serenayang/Desktop/DSC478/Homework 4"

## Question 1:

### For this problem you will use a modified version of the item-based recommender algorithm from Ch. 14 of Machine Learning in Action and use it on joke ratings data based on Jester Online Joke Recommender System. The modified version of the code is provided in the module itemBasedRec.py. Most of the module will be used as is, but you will add some additional functionality.

### The data set contains two files. The file "modified_jester_data.csv" contains the ratings on 100 jokes by 1000 users (each row is a user profile). The ratings have been normalized to be between 1 and 21 (a 20-point scale), with 1 being the lowest rating. A zero indicated a missing rating. The file "jokes.csv" contains the joke ids mapped to the actual text of the jokes.

### Your tasks in this problem are the following (please also see comments for the function stubs in the provided module):

In [2]:
from numpy import *
from numpy import linalg as la
import numpy as np

#### a. Load in the joke ratings data and the joke text data into appropriate data structures.

In [3]:
#load joke ratings data
#genfromtxt can use on missing value
ratings = genfromtxt(path + '/jokes/modified_jester_data.csv', delimiter = ',')
ratings.shape

(1000, 100)

In [4]:
ratings

array([[ 3.18, 19.79,  1.34, ...,  0.  ,  0.  ,  0.  ],
       [15.08, 10.71, 17.36, ..., 11.34,  6.68, 12.07],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [16.58, 16.63, 15.85, ...,  0.  ,  0.  ,  0.  ],
       [ 3.67,  4.45,  3.67, ...,  3.77,  3.77,  3.28],
       [ 9.88, 11.73,  9.16, ...,  0.  ,  0.  ,  0.  ]])

In [5]:
#load joke text data
def load_jokes(file):
    jokes = genfromtxt(file, delimiter=',', dtype=str)
    jokes = np.array(jokes[:,1])
    return jokes

In [6]:
def get_joke_text(jokes, id):
    return jokes[id]

In [7]:
jokes = load_jokes(path + '/jokes/jokes.csv')
jokes.shape

(100,)

In [8]:
jokes[1]

'This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That\'s an awfully big word for a ten year old."'

In [9]:
#import functions from itemBasedRec.py
def ecludSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item]>0, \
                                      dataMat[:,j]>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal
    
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    data=mat(dataMat)
    U,Sigma,VT = la.svd(data)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = data.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = data[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

# This function is not needed for Assignment 4, but may be useful for experimentation
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1] #find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

# This function performs evaluatoin on a single user based on the test_ratio
# For example, with test_ratio = 0.2, a randomly selected 20 percent of rated 
# items by the user are withheld and the rest are used to estimate the withheld ratings

def cross_validate_user(dataMat, user, test_ratio, estMethod=standEst, simMeas=pearsSim):
    number_of_items = np.shape(dataMat)[1]
    rated_items_by_user = np.array([i for i in range(number_of_items) if dataMat[user,i]>0])
    test_size = test_ratio * len(rated_items_by_user)
    test_indices = np.random.randint(0, len(rated_items_by_user), int(test_size))
    withheld_items = rated_items_by_user[test_indices]
    original_user_profile = np.copy(dataMat[user])
    dataMat[user, withheld_items] = 0 # So that the withheld test items is not used in the rating estimation below
    error_u = 0.0
    count_u = len(withheld_items)

    # Compute absolute error for user u over all test items
    for item in withheld_items:
        # Estimate rating on the withheld item
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        error_u = error_u + abs(estimatedScore - original_user_profile[item])

    # Now restore ratings of the withheld items to the user profile
    for item in withheld_items:
        dataMat[user, item] = original_user_profile[item]

    # Return sum of absolute errors and the count of test cases for this user
    # Note that these will have to be accumulated for each user to compute MAE
    return error_u, count_u


#### b. Complete the definition for the function "test". This function iterates over all users and for each performs evaluation (by calling the provided "cross_validate_user" function), and returns the error information necessary to compute Mean Absolute Error (MAE). Use this function to perform evaluation (wiht 20% test-ratio for each user) comparing MAE results using standard item-based collaborative filtering (based on the rating prediction function "standEst") with results using the SVD-based version of the rating item-based CF (using "svdEst" as the prediction engine).

In [10]:
def test(dataMat, test_ratio, estMethod):
    # Write this function to iterate over all users and for each perform evaluation by calling
    total_error = 0
    total_count = 0
    
    for i in range(len(dataMat)):
        error_u, count_u = cross_validate_user(dataMat, i, test_ratio, estMethod, simMeas=pearsSim)
        total_error += error_u
        total_count += count_u
    # the above cross_validate_user function on each user. MAE will be the ratio of total error 
    # across all test cases to the total number of test cases, for all users
    MAE = total_error / total_count
    
    print ('Mean Absoloute Error for ',estMethod,' : ',str(MAE))

In [11]:
mae_standEst = test(ratings, 0.2, standEst)
mae_standEst

Mean Absoloute Error for  <function standEst at 0x7fbd35c72ca0>  :  3.725186962735859


In [14]:
mae_svdEst = test(ratings, 0.2, svdEst)
mae_svdEst

Mean Absoloute Error for  <function svdEst at 0x7fbd35c72b80>  :  3.635666770954332


#### c. Write a new function "print_most_similar_jokes" which takes the joke ratings data, a query joke id, a parameter k for the number of nearest neighbors, and a similarity metric function, and prints the text of the query joke as well as the texts of the top k most similar jokes based on user ratings.

In [186]:
def print_most_similar_jokes(dataMat, jokes, queryJoke, k, metric=pearsSim):
    # Write this function to find the k most similar jokes (based on user ratings) to a queryJoke
    # The queryJoke is a joke id as given in the 'jokes.csv' file (an corresponding to the a column in dataMat)
    # You must compare ratings for the queryJoke (the column in dataMat corresponding to the joke), to all
    # other joke rating vectors and return the top k. Note that this is the same as performing KNN on the 
    # columns of dataMat. The function must retrieve the text of the joke from 'jokes.csv' file and print both
    # the queryJoke text as well as the text of the returned jokes.
    #Joke = dataMatT[queryJoke]
    dataMatT = dataMat.T
    temp = []
    totalSim = np.zeros((len(dataMatT), 1))
    sim = np.zeros((len(dataMatT), 1))
                        
    for i in range(len(dataMatT)):
        similarity = metric(dataMatT[queryJoke], dataMatT[i])
        if similarity == 1:
            totalSim[i] = 0
            sim[i] = i
        else:
            totalSim[i] = similarity
            sim[i] = i

    final = np.concatenate((totalSim, sim), axis =1)
    sortedF = np.flip(final[final[:,0].argsort()])
    Joke = sortedF[:, 0].astype(int)
    
    for j in Joke[:k]:
        temp.append(j)
    
    print("Selected joke: ")
    print(jokes[queryJoke] + '\n')
    print("Top %d Recommended jokes are :" %k)
    for i in temp:
        print(jokes[i])
        print("_______________")
        
        

In [187]:
print_most_similar_jokes(ratings, jokes, 3, 5, pearsSim)

Selected joke: 
Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.

Top 5 Recommended jokes are :
What do you get when you run over a parakeet with a lawnmower? Shredded tweet.
_______________
A country guy goes into a city bar that has a dress code and the maitred' demands he wear a tie. Discouraged the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant but says to the guy "Okay you're a pretty resourceful fellow you can come in... but just don't start anything"!
_______________
Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.
_______________
What do you call an American in the finals of the world cup?"Hey Beer Man!"
_______________
Q: What's the difference between a Lawyer and a Plumber? A: A Plumber works to un

## Question 2:

### For this problem you will use an image segmentation data set for clustering. You will experiment with using PCA as an approach to reduce dimensionality and noise in the data. You will compare the results of clustering the data with and without PCA using the provided image class assignments as the ground truth. The data set is divided into three files. The file "segmentation_data.txt" contains data about images with each line corresponding to one image. Each image is represented by 19 features (these are the columns in the data and correspond to the feature names in the file "segmentation_names.txt". The file "segmentation_classes.txt" contains the class labels (the type of image) and a numeric class label for each of the corresponding images in the data file. After clustering the image data, you will use the class labels to measure completeness and homogeneity of the generated clusters. The data set used in this problem is based on the Image Segmentation data set at the UCI Machine Learning Repository.

### Your tasks in this problem are the following:

#### a. Load in the image data matrix (with rows as images and columns as features). Also load in the numeric class labels from the segmentation class file. Using your favorite method (e.g., sklearn's min-max scaler), perform min-max normalization on the data matrix so that each feature is scaled to [0,1] range.

In [244]:
#Load in the image data matrix (with rows as images and columns as features).
import pandas as pd
data = pd.read_table(path + '/segmentation_data/segmentation_data.txt' ,sep=',',header = None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,110.00,189.00,9,0.00,0.00,1.00,0.67,1.22,1.19,12.93,10.89,9.22,18.67,-6.11,-11.11,17.22,18.67,0.51,1.91
1,86.00,187.00,9,0.00,0.00,1.11,0.72,1.44,0.75,13.74,11.67,10.33,19.22,-6.22,-10.22,16.44,19.22,0.46,1.94
2,225.00,244.00,9,0.00,0.00,3.39,2.20,3.00,1.52,12.26,10.33,9.33,17.11,-5.78,-8.78,14.56,17.11,0.48,1.99
3,47.00,232.00,9,0.00,0.00,1.28,1.25,1.00,0.89,12.70,11.00,9.00,18.11,-5.11,-11.11,16.22,18.11,0.50,1.88
4,97.00,186.00,9,0.00,0.00,1.17,0.69,1.17,1.01,15.59,13.89,11.78,21.11,-5.11,-11.44,16.56,21.11,0.44,1.86


In [245]:
#Also load in the numeric class labels from the segmentation class file.
labels = pd.read_table(path + '/segmentation_data/segmentation_classes.txt', header = None)
labels.head()

,0,1
0,GRASS,0
1,GRASS,0
2,GRASS,0
3,GRASS,0
4,GRASS,0


In [246]:
names = pd.read_table(path + '/segmentation_data/segmentation_names.txt', header = None)
names = list(names[0])
names

['REGION-CENTROID-COL',
 'REGION-CENTROID-ROW',
 'REGION-PIXEL-COUNT',
 'SHORT-LINE-DENSITY-5',
 'SHORT-LINE-DENSITY-2',
 'VEDGE-MEAN',
 'VEDGE-SD',
 'HEDGE-MEAN',
 'HEDGE-SD',
 'INTENSITY-MEAN',
 'RAWRED-MEAN',
 'RAWBLUE-MEAN',
 'RAWGREEN-MEAN',
 'EXRED-MEAN',
 'EXBLUE-MEAN',
 'EXGREEN-MEAN',
 'VALUE-MEAN',
 'SATURATION-MEAN',
 'HUE-MEAN']

In [247]:
#Using your favorite method (e.g., sklearn's min-max scaler), perform min-max normalization on the data matrix so 
#that each feature is scaled to [0,1] range.
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler().fit(data)
data_n = min_max_scaler.transform(data)
data_n[0:5]

array([[4.30830040e-01, 7.41666667e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.42205474e-02, 6.72233922e-04, 2.73291926e-02,
        8.55743510e-04, 9.01110284e-02, 7.94165331e-02, 6.11192912e-02,
        1.30943107e-01, 7.31343290e-01, 1.41176540e-02, 8.72865267e-01,
        1.23711348e-01, 5.08138840e-01, 8.31849232e-01],
       [3.35968379e-01, 7.33333333e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.80228046e-02, 7.26095734e-04, 3.22981359e-02,
        5.41219947e-04, 9.57913810e-02, 8.50891441e-02, 6.84830672e-02,
        1.34840205e-01, 7.29477615e-01, 2.35294199e-02, 8.59582565e-01,
        1.27393216e-01, 4.63329080e-01, 8.36986460e-01],
       [8.85375494e-01, 9.70833333e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.15969577e-01, 2.21344355e-03, 6.70807367e-02,
        1.09658970e-03, 8.54634659e-02, 7.53646732e-02, 6.18556741e-02,
        1.20031165e-01, 7.36940304e-01, 3.88235327e-02, 8.27324481e-01,
        1.13402054e-01

#### b. Next, Perform Kmeans clustering (for this problem, use the Kmeans implementation in scikit-learn) on the image data (since there are a total 7 pre-assigned image classes, you should use K = 7 in your clustering). Use Euclidean distance as your distance measure for the clustering. Print the cluster centroids (use some formatting so that they are visually understandable). Compare your 7 clusters to the 7 pre-assigned classes by computing the Completeness and Homogeneity values of the generated clusters.

In [248]:
# Perform Kmeans clustering (for this problem, use the Kmeans implementation in scikit-learn) on the image data 
#(since there are a total 7 pre-assigned image classes, you should use K = 7 in your clustering). 
from sklearn.cluster import KMeans
kmean = KMeans(n_clusters = 7, max_iter=500, verbose=1)
kmean.fit(data_n)

Initialization complete
Iteration 0, inertia 687.3919646585911
Iteration 1, inertia 390.2340074923927
Iteration 2, inertia 380.2009046272517
Iteration 3, inertia 374.91656856941404
Iteration 4, inertia 371.7750871048173
Iteration 5, inertia 370.21319634906206
Iteration 6, inertia 369.0569608466499
Iteration 7, inertia 368.37732886676775
Iteration 8, inertia 367.9492839743955
Iteration 9, inertia 367.74335080648626
Iteration 10, inertia 367.64264004709867
Iteration 11, inertia 367.5857048730359
Iteration 12, inertia 367.5783347765411
Iteration 13, inertia 367.5713177682432
Iteration 14, inertia 367.56621325336096
Iteration 15, inertia 367.5597457933926
Converged at iteration 15: strict convergence.
Initialization complete
Iteration 0, inertia 602.9575095972283
Iteration 1, inertia 397.9005447102638
Iteration 2, inertia 387.89488636428524
Iteration 3, inertia 380.18405634863933
Iteration 4, inertia 374.1412269284418
Iteration 5, inertia 370.3921940499043
Iteration 6, inertia 369.17526523

KMeans(max_iter=500, n_clusters=7, verbose=1)

In [249]:
# Use Euclidean distance as your distance measure for the clustering. Print the cluster centroids 
#(use some formatting so that they are visually understandable).

clusters = kmean.predict(data_n)

In [250]:
pd.options.display.float_format='{:,.2f}'.format

centroids = pd.DataFrame(kmean.cluster_centers_, columns = names)
centroids

,REGION-CENTROID-COL,REGION-CENTROID-ROW,REGION-PIXEL-COUNT,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
0,0.25,0.46,0.00,0.03,0.01,0.04,0.00,0.03,0.00,0.03,0.02,0.04,0.02,0.77,0.22,0.51,0.04,0.80,0.18
1,0.54,0.15,0.00,0.03,0.00,0.03,0.00,0.03,0.00,0.82,0.78,0.89,0.79,0.27,0.67,0.29,0.89,0.21,0.13
2,0.26,0.39,0.00,0.07,0.02,0.08,0.00,0.06,0.00,0.15,0.14,0.19,0.12,0.72,0.34,0.36,0.19,0.41,0.20
3,0.51,0.81,0.00,0.08,0.01,0.05,0.00,0.05,0.00,0.11,0.09,0.09,0.14,0.68,0.08,0.82,0.13,0.41,0.89
4,0.77,0.43,0.00,0.01,0.02,0.04,0.00,0.02,0.00,0.04,0.04,0.06,0.03,0.78,0.22,0.49,0.06,0.54,0.24
5,0.30,0.53,0.00,0.05,0.05,0.10,0.01,0.08,0.01,0.40,0.37,0.47,0.35,0.50,0.57,0.21,0.47,0.30,0.16
6,0.75,0.53,0.00,0.04,0.04,0.11,0.02,0.11,0.02,0.30,0.28,0.35,0.27,0.59,0.45,0.31,0.35,0.30,0.16


In [251]:
clusters

array([3, 3, 3, ..., 4, 4, 2], dtype=int32)

In [252]:
#Compare your 7 clusters to the 7 pre-assigned classes by computing the Completeness and 
#Homogeneity values of the generated clusters.
from sklearn.metrics import completeness_score, homogeneity_score

completeness = completeness_score(labels[1], clusters)
homoScore = homogeneity_score(labels[1],clusters)

print('The Completeness of Clusters: '+  str(completeness))
print ('The homogeneity of Clusters : ' + str(homoScore))

The Completeness of Clusters: 0.613187012485301
The homogeneity of Clusters : 0.6115021163370862


#### c. Perform PCA on the normalized image data matrix. You may use the linear algebra package in Numpy or the Decomposition module in scikit-learn (the latter is much more efficient). Analyze the principal components to determine the number, r, of PCs needed to capture at least 95% of variance in the data. Then use these r components as features to transform the data into a reduced dimension space. 

In [261]:
from sklearn import decomposition

pca = decomposition.PCA(n_components = 6)
DTtrans = pca.fit(data_n).transform(data_n)
np.set_printoptions(precision=2,suppress=True)

In [262]:
print(pca.explained_variance_ratio_)
print ('variance in the data: ', pca.explained_variance_ratio_.sum(0))

[0.61 0.13 0.1  0.05 0.04 0.02]
variance in the data:  0.9411392197960624


In [259]:
pca = decomposition.PCA(n_components = 7)
DTtrans = pca.fit(data_n).transform(data_n)
np.set_printoptions(precision=2,suppress=True)

In [260]:
print(pca.explained_variance_ratio_)
print ('variance in the data: ', pca.explained_variance_ratio_.sum(0))

[0.61 0.13 0.1  0.05 0.04 0.02 0.02]
variance in the data:  0.9600589227704956


In [263]:
#therefore we chose n = 7 for at least 95% of variance in the data which we got 96%.

#### d. Perform Kmeans again, but this time on the lower dimensional transformed data. Then, compute the Completeness and Homogeneity values of the new clusters.

In [267]:
from sklearn.cluster import KMeans
kmean = KMeans(n_clusters = 7)
kmean.fit(DTtrans)

KMeans(n_clusters=7)

In [268]:
print(kmean.labels_)

[3 3 3 ... 6 6 4]


In [272]:
pd.DataFrame(kmean.cluster_centers_.T, index = ['PC 1', 'PC 2', 'PC 3', 'PC 4', 'PC 5', 'PC 6'], columns = ['Cluster 1', 'Cluster 2','Cluster 3','Cluster 4','Cluster 5',' Cluster 6','Cluster 7'])

,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7
PC 1,-0.60,1.41,0.18,-0.62,-0.20,0.44,-0.51
PC 2,-0.36,0.09,0.05,0.64,-0.24,-0.11,-0.07
PC 3,0.11,0.04,-0.27,0.20,0.15,0.17,-0.34
PC 4,-0.13,-0.17,0.19,-0.09,0.06,0.24,-0.07
PC 5,-0.13,-0.03,0.03,-0.06,0.13,-0.05,0.07
PC 6,-0.02,-0.01,0.02,0.01,-0.01,-0.01,0.01


In [273]:
completeness = completeness_score(labels[1], kmean.labels_)
homoScore = homogeneity_score(labels[1],kmean.labels_)

print('The Completeness of Clusters: '+  str(completeness))
print ('The homogeneity of Clusters : ' + str(homoScore))

The Completeness of Clusters: 0.6096968341816197
The homogeneity of Clusters : 0.6080039254811925


#### e. Discuss your observations based on the comparison of the two clustering results.

In [ ]:
#Without PCA:
#The Completeness of Clusters: 0.613187012485301
#The homogeneity of Clusters : 0.6115021163370862

#With PCA dimension reduction:
#The Completeness of Clusters: 0.6096968341816197
#The homogeneity of Clusters : 0.6080039254811925

In [ ]:
#By comparing these two sets of results, we can see they are very similar between with PCA
#and without PCA dimension reduction. The scores for both completeness and homoheneity 
#are slightl better with the lower dimension transformed data. Using PCA to find the number of 
#perinciple components and then use those components to transform the data into a lower dimensionl 
#space, the results were actually impoved little.